In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
import numpy as np
import pandas as pd
import pathlib
import os
from os import getcwd
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import multiprocessing
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [2]:
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
train_dir=r'C:\data\CV\dogs-vs-cats\train_data'
val_dir=r'C:\data\CV\dogs-vs-cats\validation_data'

## tf.data.Dataset.list_files converts filrd  to the Dataset object

In [4]:

train_files = tf.data.Dataset.list_files(str(train_dir + '\\*\\*'), shuffle=False)
val_files = tf.data.Dataset.list_files(str(val_dir + '\\*\\*'), shuffle=False)


In [5]:
train_num_files=len([file for file in glob(str(train_dir + '\\*\\*'))])
val_num_files=len([file for file in glob(str(val_dir + '\\*\\*'))])
print(train_num_files)
print(val_num_files)

20000
5000


## Interleave is used to parallelize the data loading step by interleaving the contents of the datasets 
__The number of datasets to overlap can be specified by the cycle_length argument, __

__while the level of parallelism can be specified by the num_parallel_calls argument.__

## Parallelize the extraction of the stored TFRecords of the cats_vs_dogs dataset by using the interleave operation 

In [6]:
#val_size = int(num_files * 0.2)
#train_dataset = files.skip(val_size)
#train_dataset= train_files.take(train_num_files)
#val_dataset = val_files.take(val_num_files)

In [7]:
class_names = np.array(sorted([dir1 for dir1 in os.listdir(train_dir)]))
class_names

array(['cat', 'dog'], dtype='<U3')

In [8]:
epoch=10
batch_size = 32
img_height = 224
img_width = 224

In [9]:
#To process the label
def get_label(file_path):
  # convert the path to a list of path components separated by sep
  parts = tf.strings.split(file_path, os.path.sep)
  
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
# Integer encode the label
  return tf.argmax(tf.cast(one_hot, tf.int32))
# To process the image
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  
  # resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])
# To create the single training of validation example with image and its corresponding label

def process_TL(file_path):
   
  label = get_label(file_path)
  print(label)
# load the raw data from the file as a string
  img = tf.io.read_file(file_path)
 
  img = decode_img(img)
  #img= tf.image.rot90(img)
  #img=tf.image.flip_up_down(img)
  #img=tf.image.adjust_brightness(img,delta=0.1)
  img = preprocess_input(img)
  return img, label

In [10]:
train_dataset = train_files.interleave(lambda x: tf.data.Dataset.list_files(str(train_dir + '\\*\\*'), shuffle=True), cycle_length=4).map(process_TL, num_parallel_calls=tf.data.experimental.AUTOTUNE)

Tensor("ArgMax:0", shape=(), dtype=int64)


In [11]:
val_dataset = val_files.interleave(lambda x: tf.data.Dataset.list_files(str(val_dir + '\\*\\*'), shuffle=False), cycle_length=4).map(process_TL, num_parallel_calls=tf.data.experimental.AUTOTUNE)

Tensor("ArgMax:0", shape=(), dtype=int64)


In [12]:
#train_dataset= train_files.interleave(tf.data.TFRecordDataset,cycle_length=4, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#val_dataset= val_files.interleave(tf.data.TFRecordDataset,cycle_length=4, num_parallel_calls=tf.data.experimental.AUTOTUNE)

## Parallelize Transformation
__You can now apply the load_images function to each item in the train_dataset by using the map method.__

__You can parallelize the transformation of the train_dataset by using the map method with the num_parallel_class set to AUTOTUNE__

In [13]:
#train_dataset= train_dataset.interleave(tf.data.TFRecordDataset,cycle_length=4, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#val_dataset= val_dataset.interleave(tf.data.TFRecordDataset,cycle_length=4, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [14]:
#train_dataset = train_dataset.map(train_process_TL, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#val_dataset = val_dataset.map(val_process_TL, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [15]:
val_dataset.take(1)

<TakeDataset shapes: ((224, 224, 3), ()), types: (tf.float32, tf.int64)>

In [16]:
for image, label in train_dataset.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (224, 224, 3)
Label:  1


In [17]:
##Cache the dataset in-memory
#train_dataset = train_dataset.cache()
#val_dataset = val_dataset.cache()

## Parallelize Loading

In [18]:
# Shuffle and batch the train_dataset. Use a buffer size of 1024
# for shuffling and a batch size 32 for batching. 
train_dataset = train_dataset.repeat().shuffle(buffer_size=32 ).batch(batch_size) 
#val_dataset = val_dataset.shuffle(buffer_size=1024 ).batch(batch_size) 
val_dataset = val_dataset.batch(batch_size) 

In [19]:
# Parallelize the loading by prefetching the train_dataset.
# Set the prefetching buffer size to tf.data.experimental.AUTOTUNE.
train_dataset =train_dataset.prefetch(tf.data.experimental.AUTOTUNE )
val_dataset =val_dataset.prefetch(tf.data.experimental.AUTOTUNE )

In [20]:
data_augmentation = tf.keras.Sequential([
                      tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
                      tf.keras.layers.experimental.preprocessing.RandomFlip('vertical'),
                      tf.keras.layers.experimental.preprocessing.RandomRotation(0.45),
                      tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
                      tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
                     
                    ])

In [21]:
def create_model():
    input_layer = tf.keras.layers.Input(shape=(224, 224, 3))
    x= data_augmentation(input_layer)
    base_model = tf.keras.applications.MobileNetV2(input_tensor=x,
                                                   weights='imagenet',
                                                   include_top=False)
    
    base_model.trainable = False
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = tf.keras.layers.Dense(2, activation='softmax')(x)
    
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [22]:
model= create_model()


In [23]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           sequential[0][0]                 
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
_______________________________________________________________________________________

In [24]:
from datetime import datetime
# Create a TensorBoard callback
log_dir ='C:\\Users\\khandelwalr\\Python_Code\\TF_data\\logs\\' + datetime.now().strftime('%Y%m%d-%H%M%S')
print(log_dir)


tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

C:\Users\khandelwalr\Python_Code\TF_data\logs\20200928-062900


In [25]:
# Include the epoch in the file name (uses `str.format`)

model_name='TF_data'
# best checkpoint
filename = model_name + '.h5'
checkpoint_path = os.path.join('model_save', filename)

# Create a callback that saves the model's weights every 5 epochs
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
                                                 monitor='val_accuracy', 
                                                 verbose=1,
                                                 save_freq='epoch',
                                                 save_best_only=True, 
                                                 save_weights_only=False, 
                                                 mode='max')

In [26]:
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["val_accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

In [27]:
my_callback = MyThresholdCallback(threshold=0.99)

In [28]:
import time
start_time= time.perf_counter()
history_tfdata =model.fit(train_dataset,
          steps_per_epoch=int((train_num_files)/batch_size),
          validation_data=   val_dataset,
          validation_steps=int(val_num_files/batch_size),
                           callbacks=[my_callback],
         
          epochs=epoch)

print(time.perf_counter()-start_time)


Epoch 1/10
625/625 [==============================] - 29s 46ms/step - loss: 0.1958 - accuracy: 0.9183 - val_loss: 0.0928 - val_accuracy: 0.9671
Epoch 2/10
625/625 [==============================] - 29s 46ms/step - loss: 0.1621 - accuracy: 0.9345 - val_loss: 0.0957 - val_accuracy: 0.9679
Epoch 3/10
625/625 [==============================] - 29s 46ms/step - loss: 0.1574 - accuracy: 0.9370 - val_loss: 0.0924 - val_accuracy: 0.9696
Epoch 4/10
625/625 [==============================] - 29s 46ms/step - loss: 0.1568 - accuracy: 0.9368 - val_loss: 0.0497 - val_accuracy: 0.9856
Epoch 5/10
625/625 [==============================] - 29s 46ms/step - loss: 0.1421 - accuracy: 0.9459 - val_loss: 0.0856 - val_accuracy: 0.9752
Epoch 6/10
625/625 [==============================] - 29s 46ms/step - loss: 0.1463 - accuracy: 0.9416 - val_loss: 0.0919 - val_accuracy: 0.9712
Epoch 7/10
625/625 [==============================] - 29s 46ms/step - loss: 0.1442 - accuracy: 0.9405 - val_loss: 0.0587 - val_accuracy:

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

from tensorflow.keras.models import load_model

from tensorflow.keras import optimizers, callbacks


In [39]:
image_gen_train = ImageDataGenerator(rescale=1./255, 
                                     zoom_range=0.1, 
                                     rotation_range=45,
                                     shear_range=0.1,
                                     horizontal_flip=True,
                                     vertical_flip=True)
# image_gen_train = ImageDataGenerator(rescale=1./255)

In [40]:
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(224,224),
                                                     class_mode='sparse')

Found 20000 images belonging to 2 classes.


In [41]:
image_gen_val = ImageDataGenerator(rescale=1./255)

In [42]:
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=val_dir,
                                                 target_size=(224,224),
                                                 class_mode='sparse')

Found 5000 images belonging to 2 classes.


In [43]:
len(train_data_gen)

625

In [44]:
def create_model():
    input_layer = tf.keras.layers.Input(shape=(224, 224, 3))
    input_layer=preprocess_input(input_layer)
   
    base_model = tf.keras.applications.MobileNetV2(input_tensor=input_layer,
                                                   weights='imagenet',
                                                   include_top=False)
    
    base_model.trainable = False
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = tf.keras.layers.Dense(2, activation='softmax')(x)
    
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [45]:
model_idg=create_model()

In [47]:
start_time2= time.perf_counter()
history = model_idg.fit(
    train_data_gen,
    steps_per_epoch=len(train_data_gen),
    epochs=10,
callbacks=[my_callback],
    validation_data=val_data_gen,
    validation_steps=len(val_data_gen)
)
print(time.perf_counter()-start_time2)

Epoch 1/10
625/625 [==============================] - 263s 420ms/step - loss: 0.2053 - accuracy: 0.9147 - val_loss: 0.0757 - val_accuracy: 0.9724
Epoch 2/10
625/625 [==============================] - 260s 415ms/step - loss: 0.1690 - accuracy: 0.9311 - val_loss: 0.0693 - val_accuracy: 0.9724
Epoch 3/10
625/625 [==============================] - 259s 415ms/step - loss: 0.1638 - accuracy: 0.9338 - val_loss: 0.0606 - val_accuracy: 0.9794
Epoch 4/10
625/625 [==============================] - 258s 413ms/step - loss: 0.1535 - accuracy: 0.9376 - val_loss: 0.0632 - val_accuracy: 0.9770
Epoch 5/10
625/625 [==============================] - 257s 411ms/step - loss: 0.1529 - accuracy: 0.9385 - val_loss: 0.0641 - val_accuracy: 0.9766
Epoch 6/10
625/625 [==============================] - 259s 414ms/step - loss: 0.1531 - accuracy: 0.9383 - val_loss: 0.0568 - val_accuracy: 0.9804
Epoch 7/10
625/625 [==============================] - 258s 414ms/step - loss: 0.1539 - accuracy: 0.9371 - val_loss: 0.0605 -

In [ ]:
val_eval_files = tf.data.Dataset.list_files(str(val_dir + '\\*\\*'), shuffle=False)
val_eval_num_files=len([file for file in glob(str(val_dir + '\\*\\*'))])
#val_eval_dataset = val_eval_files.take(val_eval_num_files)
val_eval_dataset = val_files.interleave(lambda x: tf.data.Dataset.list_files(str(val_dir + '\\*\\*'), shuffle=False), cycle_length=4).map(process_TL, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_eval_dataset =val_eval_dataset.batch(batch_size)
val_dataset =val_dataset.prefetch(tf.data.experimental.AUTOTUNE )
loss, accuracy = model.evaluate(val_eval_dataset)
print("Loss :", loss)
print("Accuracy :", accuracy)